In [1]:
# make sure notebook plots inline
%matplotlib inline

# first import packages
import matplotlib.pylab as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi']= 150
import numpy as np

import sys
sys.path.append('..')
import tidy3d as td